In [1]:
import numpy as np
import cv2
import winsound

In [6]:
# yolo 모델 읽어오기
with open('coco.names.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]   
    colors = np.random.uniform(0,255,size=(len(classes),3))

In [9]:
yolo_model = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
layer_names = yolo_model.getLayerNames()
out_layers = [layer_names[i-1] for i in yolo_model.getUnconnectedOutLayers()]

In [15]:
def process_video():
    video = cv2.VideoCapture(0)
    while video.isOpened():
        success,img = video.read()
        if success:
            height,width,channels = img.shape
            blob = cv2.dnn.blobFromImage(img,1.0/255,(448,448),(0,0,0),swapRB=True,crop=False)
            yolo_model.setInput(blob)
            outputs = yolo_model.forward(out_layers)
            
            # 확률이 0.5를 넘는 바운딩 박스를 생성하고 모음
            class_ids,confidences,boxes=[],[],[]
            for output in outputs:
                for vec85 in output:
                    scores=vec85[5:]
                    class_id=np.argmax(scores)
                    confidence=scores[class_id]
                    if confidence>0.5: # 신뢰도가 50% 이상인 경우만 취함
                        centerx,centery=int(vec85[0]*width),int(vec85[1]*height)  # [0,1] 표현을 영상 크기로 변환
                        w,h=int(vec85[2]*width),int(vec85[3]*height)
                        x,y=int(centerx-w/2),int(centery-h/2)
                        boxes.append([x,y,w,h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)  #분류된것중에 최대인 것만 남김
            
            # 살아남은 바운딩 박스를 영상에 표시
            for i in range(len(boxes)):
                if i in indexes:
                    x,y,w,h=boxes[i]
                    text=str(classes[class_ids[i]])+'%.3f'%confidences[i]
                    cv2.rectangle(img,(x,y),(x+w,y+h),colors[class_ids[i]],2)
                    cv2.putText(img,text,(x,y+30),cv2.FONT_HERSHEY_PLAIN,2,colors[class_ids[i]],2)
            cv2.imshow('object detecting',img)
            
            if 0 in class_ids:
                print("도둑이 들었습니다.")
                winsound.Beep(frequency=2000,duration=500)                
            
        key = cv2.waitKey(1) & 0XFF
        if key == 27 :break
    video.release()
    cv2.destroyAllWindows()

In [16]:
process_video()

도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.
도둑이 들었습니다.


In [14]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']